In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
import os 
import PIL
import glob
import torch
import torchvision
import numpy as np
import matplotlib.pyplot as plt
from torchvision import transforms
from tqdm import tqdm_notebook as tqdm
from source.network import NNClassifier

from source.helpme import get_gesture_dataset, load_imgs_from_folder, show_image, calculate_pad, show_history

In [ ]:
X, y = get_gesture_dataset(size=(64, 64), shuffle=True)

X.shape, y.shape

In [ ]:
index = 150
show_image(X[index], title=y[index])

# Convolutional neural network

In [ ]:
class Softmax_layer(torch.nn.Module):
    def __init__(self):
        super().__init__()
    
    def forward(self, x):
        e = torch.exp(x - x.max(1, True)[0] )
        summ = e.sum(1, True)[0]
        return e / summ
    
class Flatten(torch.nn.Module):
    def forward(self, x):
        N = x.shape[0]
        return x.view(N, -1)

In [ ]:
calculate_pad(input_size = 64, kernel_size = 3, stride = 2, output_size = 32)

In [ ]:
conv_net = torch.nn.Sequential(torch.nn.Conv2d(1, 8, 3, stride=2, padding=1),
                               torch.nn.ReLU(),
                               torch.nn.BatchNorm2d(8),
                               # 32x32x16
                               
                               torch.nn.Conv2d(8, 16, 3, stride=2, padding=1),
                               torch.nn.ReLU(),
                               # 16x16x32
                               
                               torch.nn.Conv2d(16, 32, 3, stride=2, padding=1),
                               torch.nn.ReLU(),
                               torch.nn.BatchNorm2d(32),
                               # 8x8x64
                               
                               torch.nn.Conv2d(32, 64, 3, stride=2, padding=1),
                               torch.nn.ReLU(),
                               # 4x4x64
                               
                               Flatten(),
                               torch.nn.Linear(1024, 10),
                               Softmax_layer())

In [ ]:
model = NNClassifier(conv_net, lr=5e-4, device='cpu')

In [ ]:
train_size = 1400
valid_size = 250
test_size = len(X) - train_size - valid_size
print('train size =', train_size)
print('valid size =', valid_size)
print('test size =', test_size)

X_train = X[ : train_size]
y_train = y[ : train_size]

X_valid = X[train_size : train_size+valid_size]
y_valid = y[train_size : train_size+valid_size]

X_test = X[train_size+valid_size : ]
y_test = y[train_size+valid_size : ]

In [ ]:
model.fit(X_train, y_train, epochs=30, batch_size=256, 
          valid_data=(X_valid, y_valid), log_every_epoch=1)

In [ ]:
show_history(model.train_history, model.valid_history, width=2, fontsize=10)

In [ ]:
model.evaluate_score(X_test, y_test)

# Testing on real data

Перед тем как перейти к тестированию на реальных данных, надо положить картинки в любом формате в папку source/real_data. Все что там находится можно удалить.

In [ ]:
imgs = load_imgs_from_folder(os.path.join('source', 'real_data'), size=(64,64))

In [ ]:
imgs.shape

In [ ]:
show_image(imgs[0])